<div style="
    background: linear-gradient(135deg,rgb(0, 0, 0),rgb(5, 19, 3),rgba(21, 105, 6, 0.83));
    width: 50%;
    margin: 40px auto;
    border-radius: 25px;
    text-align: center;
    box-shadow: 0px 0px 10px rgb(64, 255, 0);
    color: #e3f2fd;
    font-size: 28px;
    padding: 25px 35px;
    font-family: 'Poppins', 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    font-weight: 600;
    letter-spacing: 1px;
    transition: all 0.3s ease;
">
    About Dataset
</div>


<div style="
    background: linear-gradient(135deg,rgb(0, 0, 0),rgb(5, 19, 3),rgba(21, 105, 6, 0.83));
    width: 30%;
    margin: 40px auto;
    border-radius: 25px;
    text-align: center;
    box-shadow: 0px 0px 10px rgb(64, 255, 0);
    color: #e3f2fd;
    font-size: 24px;
    padding: 25px 35px;
    font-family: 'Poppins', 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    font-weight: 600;
    letter-spacing: 1px;
    transition: all 0.3s ease;
">
    Preprocessing & Visualization
</div>


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#plt.style.use("seaborn")
import chardet
import csv
import itertools
import math
# Import libraries
%matplotlib inline

import pgeocode


import plotly as py


import zipcodes as zcode
import plotly.express as px


import plotly.graph_objects as go
import matplotlib.gridspec as gridspec
from plotly.subplots import make_subplots
from matplotlib.patches import ConnectionPatch
from plotly.offline import init_notebook_mode, iplot, plot
import warnings
warnings.filterwarnings('ignore')
from sklearn import metrics
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report,confusion_matrix,precision_score,jaccard_score,recall_score,f1_score

In [ ]:
data = pd.read_csv(r"C:\\Users\\SONY\\Downloads\\Bank_Personal_Loan_Modelling(1).csv")
data



In [ ]:
df=pd.DataFrame(data)
df



1. The CCAvg column needs to be converted to a number, as it is formatted incorrectly (0/30 instead of 0.30)
2. There are negative values ​​in Experience and they need to be corrected (e.g. with the median of positives)
3. The majority of customers have not taken out a personal loan (only 9.6% have) → the data is unbalanced

In [ ]:
df.describe(include="all")

we can see statistical information on the table above

In [ ]:
df.describe(include="object")

In [ ]:
df.info()

Modify the CCAvg column (average credit card usage) and modify the Experience column (work experience)

In [ ]:
# ✅ 1. Fix the CCAvg column (convert '0/30' to 0.30)
df['CCAvg'] = df['CCAvg'].str.replace("/", ".", regex=False)
df['CCAvg'] = pd.to_numeric(df['CCAvg'], errors='coerce')

# ✅ 2.positive, negative values in Experience
df["Experience"]=abs(df["Experience"])

# Check the result
print(df[['CCAvg', 'Experience']].describe())

In [ ]:
print(df[df["Experience"]<0])

In [ ]:
print(df.shape)

In [ ]:
with open("C:\\Users\\SONY\\Downloads\\Bank_Personal_Loan_Modelling(1).csv", 'rb') as f:
    result = chardet.detect(f.read())
    print(result['encoding'])

In [ ]:
with open(r"C:\\Users\\SONY\\Downloads\\Bank_Personal_Loan_Modelling(1).csv", 'r') as f:


    dialect = csv.Sniffer().sniff(f.read(1024))


    print(f"The delimiter detected for the car dataset is: {dialect.delimiter}")




In [ ]:
print(df.columns)

In [ ]:
def describe(df):


    variables = []   # Column names
    dtypes = []    # Data types of columns
    count = []      # Total number of entries in each column
    unique = []      # Number of unique values in each column
    missing = []    # Count of missing (NaN) values in each column

     # Loop through each column in the DataFrame
    for item in df.columns:
        variables.append(item)    # Append column name
        dtypes.append(df[item].dtype)  # Append column data type
        count.append(len(df[item]))
        unique.append(len(df[item].unique()))
        missing.append(df[item].isna().sum())


    output = pd.DataFrame({
        'Variable': variables,
        'Dtype': dtypes,
        'Count': count,
        'Unique': unique,
        'Missing value': missing
    })

    return output

In [ ]:
describe(df)

In [ ]:
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
percent = pd.DataFrame(missing_value_df)
percent

In [ ]:
# Display unique values for each column
for column in df.columns:
    print(f"Unique values in '{column}':")
    print(df[column].unique())
    print("-" * 40)

In [ ]:
print(df.nunique())

In [ ]:
for column in df.columns:
    print(f"Value counts for '{column}':")
    print(df[column].value_counts())
    print("-" * 40)

In [ ]:
df.isnull().sum()

In [ ]:
# identify numeric and categorical columns
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

# result
print("Numeric columns:")
print(numeric_cols)

print("\nCategorical columns:")
print(categorical_cols)

In [ ]:
#ccavg yearly
df["CCAvg"]*12

In [ ]:
# Drop ID column (not useful for analysis)


# Select key features for plotting
selected_features = ['Income', 'CCAvg', 'Education', 'Mortgage', 'CD Account',"Age","Experience","Family","Securities Account","Online","CreditCard"]

# Set seaborn style
sns.set(style="whitegrid")

# Use distinct colors for each feature
colors = sns.color_palette("Set3", len(selected_features))

# Draw colorful boxplots grouped by Personal Loan
for i, feature in enumerate(selected_features):
    plt.figure(figsize=(8, 5))
    sns.boxplot(data=df, x=feature, y='Personal Loan', orient='h', color=colors[i])
    plt.title(f'{feature} by Personal Loan', fontsize=14, fontweight='bold')
    plt.xlabel(feature, fontsize=12)
    plt.ylabel("Personal Loan", fontsize=12)
    plt.yticks([0, 1], ['No Loan', 'Loan'], fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()

In [ ]:
# Create individual boxplots for each numeric feature (not grouped by Personal Loan)
# Use colorful style for each plot
sns.set(style="whitegrid")
vivid_palette = sns.color_palette("Set3")

# Generate simple, colorful boxplots for each feature
for idx, col in enumerate(numeric_cols):
    if col != 'Personal Loan':
        plt.figure(figsize=(7, 4))
        sns.boxplot(x=df[col], color=vivid_palette[idx % len(vivid_palette)])
        plt.title(f'Boxplot of {col}', fontsize=13, fontweight='bold')
        plt.xlabel(col, fontsize=11)
        plt.grid(True, linestyle='--', alpha=0.5)
        plt.tight_layout()
        plt.show()

1. Age: The distribution looks normal, no major outliers.
 2. Experience: Cleaned for negative values; otherwise evenly distributed.
 3. Income: Shows clear outliers on the higher end — potential high earners.
 4. ZIP Code: Has an extreme outlier — likely a data entry error (e.g., ZIP code in the 10000 range vs. 90000s).
 5. Family: Mostly small family sizes (1-4), no outliers expected.
 6. CCAvg: Shows many outliers — some customers spend heavily on credit cards.
 7. Education: Three distinct levels as expected (no outliers).
 8. Mortgage: Heavy right skew with many high-value outliers.
 9. Securities Account, CD Account, Online, CreditCard: These are binary (0/1), so the boxplots just show two values with very limited spread — not useful for distribution but good for checking presence

In [ ]:
sns.boxplot(x=df['Family'],y=df['Income'],hue=df['Personal Loan'], palette='hot')

In [ ]:
sns.boxplot(x="Education", y='Mortgage', hue="Personal Loan", data=df,palette="hot")
plt.legend(title='Personal Loan', loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
sns.boxplot(x='Education',y= 'Income' ,hue= 'Personal Loan', data=df,palette="hot")
plt.legend(title='Personal Loan', loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:

# Identify numeric columns (excluding binary/categorical)
binary_cols = ['CD Account', 'Securities Account', 'Online', 'CreditCard','Personal Loan']
continuous_cols = [col for col in df.select_dtypes(include='number').columns if col not in binary_cols]

# Calculate IQR, bounds, and count outliers for each continuous column
outlier_summary = []

for col in continuous_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    outlier_summary.append({
        'Feature': col,
        'Q1': Q1,
        'Q3': Q3,
        'IQR': IQR,
        'Lower Bound': lower_bound,
        'Upper Bound': upper_bound,
        'Outlier Count': len(outliers),
        'Outlier Indexes': outliers.index.tolist()
    })
# Convert to DataFrame and display
iqr_df = pd.DataFrame(outlier_summary)
print(iqr_df)

In [ ]:
numeric_cols = ['Age', 'Experience', 'Income', 'ZIP Code', 'CCAvg', 'Mortgage',"ID"]

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR


    outlier_rows = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    unique_outlier_count = outlier_rows.shape[0]
    total_rows = df.shape[0]
    percentage = (unique_outlier_count / total_rows) * 100

    print(f"Feature: {col}")
    print(f"Number of unique outlier rows: {unique_outlier_count}")
    print(f"Percentage of outliers: {percentage:.2f}%\n")

In [ ]:
# Identify discrete and continuous variables
discrete_vars = [col for col in df.columns if df[col].nunique() < 10]
continuous_vars = [col for col in df.columns if col not in discrete_vars]

# Ensure both lists have the same length
max_length = max(len(discrete_vars), len(continuous_vars))
discrete_vars += [""] * (max_length - len(discrete_vars))
continuous_vars += [""] * (max_length - len(continuous_vars))

# Create a properly formatted DataFrame
sorted_vars_df = pd.DataFrame({
    "Discrete Variables": sorted(discrete_vars),
    "Continuous Variables": sorted(continuous_vars)
})

# Display the table
print(sorted_vars_df)

In [ ]:
# Categorical columns
CatCols = ['CD Account', 'CreditCard', 'Education', 'Family',"Online","Personal Loan","Securities Account"]
df[CatCols] = df[CatCols].astype(str)

# Colors
colors = [
    ['#0055ff', '#00aaff', '#3399ff'],
    ['#ff7000', '#ffaa00'],
    ['#23bf00', '#55dd55'],
    ['#cc00cc', '#ff99ff', '#aa00aa', '#dd66dd'],
    ['#0055ff', '#00aaff'],
    ['#ff7000', '#ffaa00'],
    ['#23bf00', '#55dd55'],


]

# Plot
fig, ax = plt.subplots(nrows=1, ncols=7, figsize=(24, 6), dpi=100)

for i, col in enumerate(CatCols):
    chart = sns.countplot(x=col, data=df, ax=ax[i], palette=colors[i])
    ax[i].set_title(f'{col} Distribution', fontsize=16, pad=15)
    ax[i].set_xlabel(col, fontsize=16, labelpad=15)
    ax[i].set_ylabel('Count', fontsize=16, labelpad=10)
    ax[i].tick_params(axis='x', labelsize=13)
    ax[i].tick_params(axis='y', labelsize=12)

    # Add value labels on top of each bar
    for container in chart.containers:
        chart.bar_label(container, fontsize=14, padding=4)

# Main title with space
plt.suptitle('Frequency Distribution of Categorical Variables', fontsize=34, fontweight='bold', y=1.08)
plt.tight_layout()
plt.show()

In [ ]:
print(pd.unique(df["CD Account"]))
print(pd.unique(df["CreditCard"]))
print(pd.unique(df["Education"]))
print(pd.unique(df["Family"]))
print(pd.unique(df["Online"]))
print(pd.unique(df["Personal Loan"]))
print(pd.unique(df["Securities Account"]))




In [ ]:
NumCols = df.select_dtypes(include=['int64', 'float64']).columns

In [ ]:

colors = sns.color_palette("husl", len(NumCols))

n_cols = 3
n_rows = math.ceil(len(NumCols) / n_cols)
fig, ax = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(18, 5 * n_rows), dpi=100)
ax = ax.flatten()

for i, col in enumerate(NumCols):
    values, bin_edges = np.histogram(df[col], bins=20, range=(np.floor(df[col].min()), np.ceil(df[col].max())))
    graph = sns.histplot(data=df, x=col, bins=bin_edges, kde=True, ax=ax[i],
                         edgecolor='black', color=colors[i], alpha=0.6)

    ax[i].set_xlabel(col, fontsize=12)
    ax[i].set_ylabel('Count', fontsize=10)
    ax[i].grid(color='lightgrey')

    stats_text = '\n'.join([
        f'μ = {df[col].mean():.2f}',
        f'σ = {df[col].std():.2f}',
        f'Median = {df[col].median():.2f}',
        f'Min = {df[col].min():.2f}',
        f'Max = {df[col].max():.2f}'
    ])

    ax[i].text(0.65, 0.95, stats_text, transform=ax[i].transAxes,
               fontsize=9, verticalalignment='top',
               bbox=dict(boxstyle='round', facecolor=colors[i], edgecolor='black', pad=0.4))


for j in range(i + 1, len(ax)):
    ax[j].axis('off')

plt.suptitle('Distribution of Numerical Variables', fontsize=20)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

✅ 1. Age
 • The distribution is roughly normal with a slight left skew.
 • Most individuals are between 30 and 60 years old.
 • This variable appears well-behaved and likely suitable for modeling without major transformation.

⸻

✅ 2. CCAvg (Credit Card Average Spending)
 • The distribution is heavily right-skewed.
 • Most customers have low credit card spending, with a few individuals spending significantly more — these may be outliers.
 • A log transformation could help normalize this variable.

⸻

✅ 3. Experience
 • The distribution is somewhat uniform but not clean.

 • Needs cleaning and perhaps imputation or exclusion of invalid values.

⸻

✅ 4. ZIP Code
 • Although shown as a numerical variable, ZIP Code is categorical, not continuous.

  “ZIP Code is technically numeric – but analytically, it is not continuous and should not be         examined with numeric statistical methods.”
 .

⸻

✅ 5. Income
 • The distribution is strongly right-skewed.
 • Most customers have low to moderate income, but a few individuals have very high incomes.
 • Log transformation is advisable to reduce skewness before modeling.

⸻

✅ 6. Mortgage
 • This feature has a very sharp right skew.
 • A large number of people have zero mortgage, while only a few have large values.
 • You’re likely dealing with a zero-inflated variable. Consider transformation or binning.

⸻

🔍 Overall Summary:

Feature Distribution Type Analysis Notes
Age Roughly normal Ready for modeling
Experience Irregular / flat Check for negative values
Income Right-skewed Apply log transform
CCAvg Right-skewed Log transform or outlier handling needed
Mortgage Right-skewed + zeros Consider binning or transformation
ZIP Code Categorical (not numeric) Use for grouping onl
Note: Although ZIP Code is a numeric type, it is a categorical ID, not a continuous variable. So it makes no sense to display density or descriptive statistics on it.
So the result:
• ZIP Code is technically numeric ✅
• But analytically, it is not continuous and should not be examined with numeric statistical methods


In [ ]:

print(df['Personal Loan'].value_counts())

In [ ]:

#group by based on ZIP Code و Personal Loan
zip_loan_counts = df.groupby(['ZIP Code', 'Personal Loan']).size().unstack(fill_value=0)


zip_loan_counts.columns = ['No Loan', 'Loan']

#diplay
print(zip_loan_counts.head(10))

In [ ]:

top_zipcodes = zip_loan_counts.head(10)

# bar chart
top_zipcodes.plot(kind='bar', stacked=True, figsize=(12, 6), color=['skyblue', 'salmon'])

plt.title('Number of Loans and Non-Loans by ZIP Code', fontsize=16,color="purple")
plt.xlabel('ZIP Code', fontsize=18,color="blue")
plt.ylabel('Number of Customers', fontsize=18,color="red")
plt.legend(title='Personal Loan',fontsize=12,loc="upper right")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
#this code ,it produces an interactive map for the United States.
import pandas as pd
import pgeocode
import folium
#convert zip code to strings and remove duplicates
zip_codes = df['ZIP Code'].astype(str).unique()

# GETTING coordinates with pgeocode
nomi = pgeocode.Nominatim('us')
geo_df = nomi.query_postal_code(zip_codes)

# Delete records without coordinate
geo_df = geo_df.dropna(subset=['latitude', 'longitude'])

# create map
mean_lat = geo_df['latitude'].mean()
mean_lon = geo_df['longitude'].mean()
map_zip = folium.Map(location=[mean_lat, mean_lon], zoom_start=6)


for _, row in geo_df.iterrows():
    popup_text = f"ZIP: {row['postal_code']}<br>Lat: {row['latitude']:.4f}, Lon: {row['longitude']:.4f}"
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=4,
        color='blue',
        fill=True,
        fill_opacity=0.6,
        popup=popup_text
    ).add_to(map_zip)

# save map
map_zip.save('zip_map.html')
print("✅ Map saved as 'zip_map.html' with smaller markers.")
from IPython.display import IFrame
IFrame('zip_map.html', width=800, height=600)
map_zip


In [ ]:
#extracting zipcodes
zip_list=(df['ZIP Code'].tolist())
zip_code=[str(x) for x in zip_list]
nomi = pgeocode.Nominatim('Us')
address= nomi.query_postal_code(zip_code)
address

In [ ]:
address.drop(["community_name" ,"community_code","place_name","state_name","state_code","accuracy"]  ,axis='columns', inplace=True)
address

In [ ]:
address.isnull().sum()

In [ ]:
dff = pd.concat([df, address], axis=1)
dff

In [ ]:
# Create grouped count and percent
sex_percent = df.groupby(['CD Account', 'Personal Loan']).size().reset_index(name='count')
sex_percent['percent'] = sex_percent.groupby('CD Account')['count'].transform(lambda x: x / x.sum())

# Plot with percent
plt.figure(figsize=(6, 4))
sns.barplot(data=sex_percent, x='CD Account', y='percent', hue='Personal Loan')

# Add percentage labels on top of bars
for container in plt.gca().containers:
    plt.bar_label(container, labels=[f'{v:.0%}' for v in container.datavalues], label_type='edge')

plt.title("Personal loan Distribution by CD Account Status",fontsize=14,color="red")
plt.ylabel("Percentage",color="purple")
plt.xlabel("CD Account",color="blue")
plt.ylim(0, 1)
plt.tight_layout()
plt.show()

In [ ]:
# Create grouped count and percent
sex_percent = df.groupby(['CreditCard', 'Personal Loan']).size().reset_index(name='count')
sex_percent['percent'] = sex_percent.groupby('CreditCard')['count'].transform(lambda x: x / x.sum())

# Plot with percent
plt.figure(figsize=(6, 4))
sns.barplot(data=sex_percent, x='CreditCard', y='percent', hue='Personal Loan')

# Add percentage labels on top of bars
for container in plt.gca().containers:
    plt.bar_label(container, labels=[f'{v:.0%}' for v in container.datavalues], label_type='edge')

plt.title("Personal loan Distribution by CreditCard status",fontsize=14,color="red")
plt.ylabel("Percentage",color="purple")
plt.xlabel("CreditCard",color="blue")
plt.ylim(0, 1)
plt.tight_layout()
plt.show()

In [ ]:
# Create grouped count and percent
sex_percent = df.groupby(['Education', 'Personal Loan']).size().reset_index(name='count')
sex_percent['percent'] = sex_percent.groupby('Education')['count'].transform(lambda x: x / x.sum())

# Plot with percent
plt.figure(figsize=(6, 4))
sns.barplot(data=sex_percent, x='Education', y='percent', hue='Personal Loan')

# Add percentage labels on top of bars
for container in plt.gca().containers:
    plt.bar_label(container, labels=[f'{v:.0%}' for v in container.datavalues], label_type='edge')

plt.title("Personal loan Distribution by  Education",fontsize=14,color="red")
plt.ylabel("Percentage",color="purple")
plt.xlabel("Education",color="blue")
plt.ylim(0, 1)

plt.legend(
    title='Personal Loan',
    title_fontsize=10,
    fontsize=9,
    loc='upper right',
    bbox_to_anchor=(1, 1)
)
plt.tight_layout()
plt.show()

In [ ]:
# Create grouped count and percent
sex_percent = df.groupby(['Family', 'Personal Loan']).size().reset_index(name='count')
sex_percent['percent'] = sex_percent.groupby('Family')['count'].transform(lambda x: x / x.sum())

# Plot with percent
plt.figure(figsize=(8, 4))
sns.barplot(data=sex_percent, x='Family', y='percent', hue='Personal Loan')

# Add percentage labels on top of bars
for container in plt.gca().containers:
    plt.bar_label(container, labels=[f'{v:.0%}' for v in container.datavalues], label_type='edge')

plt.title("Personal loan Distribution by  Family ",fontsize=14,color="red")
plt.ylabel("Percentage",color="purple")
plt.xlabel("CreditCard",color="blue")
plt.ylim(0, 1)
plt.legend(
    title='Personal Loan',
    title_fontsize=10,
    fontsize=9,
    loc='upper right',
    bbox_to_anchor=(1, 1)
)
plt.tight_layout()
plt.show()

In [ ]:
# Create grouped count and percent
sex_percent = df.groupby(['Online', 'Personal Loan']).size().reset_index(name='count')
sex_percent['percent'] = sex_percent.groupby('Online')['count'].transform(lambda x: x / x.sum())

# Plot with percent
plt.figure(figsize=(8, 4))
sns.barplot(data=sex_percent, x='Online', y='percent', hue='Personal Loan')

# Add percentage labels on top of bars
for container in plt.gca().containers:
    plt.bar_label(container, labels=[f'{v:.0%}' for v in container.datavalues], label_type='edge')

plt.title("Personal loan Distribution by Online ",fontsize=14,color="red")
plt.ylabel("Percentage",color="purple")
plt.xlabel("Online",color="blue")
plt.ylim(0, 1)
plt.legend(
    title='Personal Loan',
    title_fontsize=10,
    fontsize=9,
    loc='upper right',
    bbox_to_anchor=(1, 1)
)
plt.tight_layout()
plt.show()

In [ ]:
#convert to int
cols_to_convert = ['Personal Loan', 'CD Account', 'Online', 'CreditCard',
                   'Securities Account', 'Education', 'Family']

for col in cols_to_convert:
    df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')

In [ ]:
df.Age.max()

In [ ]:
 #Proportion of Heart Disease (`target`) for Different Age Groups
df["age_group"] = pd.cut(df["Age"], bins=[20, 40, 50, 60, 70], labels=["20-40", "40-50", "50-60", "60-70"])
print("\n💓 **Proportion of `Personal Loan` for Each Age Group:**")
print(df.groupby("age_group")["Personal Loan"].value_counts(normalize=True).unstack())
numerical_df = df.select_dtypes(include=["number"])
# Countplot for `age_group` distribution across `target`
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x="age_group", hue="Personal Loan", palette=["pink", "blue"])
plt.title("📊 Distribution of `age_group` by `Personal Loan`",color="red",fontsize=14)
plt.xlabel("Age Group",color="purple")
plt.ylabel("Count",color="blue")
plt.legend(title="Personal Loan")
plt.show()


In [ ]:
Age_0_30 = []
Age_30_50 = []
Age_50_65 = []
Age_65 = []

for i in range(len(df)):
    age = df.loc[i, 'Age']
    loan = df.loc[i, 'Personal Loan']

    if loan == 1:
        if age < 30:
            Age_0_30.append(age)
        elif age < 50:
            Age_30_50.append(age)
        elif age < 65:
            Age_50_65.append(age)
        else:
            Age_65.append(age)


total_loans = len(df[df['Personal Loan'] == 1])

print(f'Number of Personal Loans age <30: {len(Age_0_30)} ({len(Age_0_30)/total_loans:.2%})')
print(f'Number of Personal Loans age 30–50: {len(Age_30_50)} ({len(Age_30_50)/total_loans:.2%})')
print(f'Number of Personal Loans age 50–65: {len(Age_50_65)} ({len(Age_50_65)/total_loans:.2%})')
print(f'Number of Personal Loans age >65: {len(Age_65)} ({len(Age_65)/total_loans:.2%})')

finding the ages of people who received loans

In [ ]:

family_stats = {
    1: {'total': 0, 'loan': 0},
    2: {'total': 0, 'loan': 0},
    3: {'total': 0, 'loan': 0},
    4: {'total': 0, 'loan': 0},
}


for index, row in df.iterrows():
    family_count = row['Family']
    loan_status = row['Personal Loan']

    if family_count in family_stats:
        family_stats[family_count]['total'] += 1
        if loan_status == 1:
            family_stats[family_count]['loan'] += 1


for fam_size in sorted(family_stats.keys()):
    total = family_stats[fam_size]['total']
    loans = family_stats[fam_size]['loan']
    percentage = loans / total * 100 if total > 0 else 0

    print(f'📌 Family size {fam_size}:')
    print(f'   - Total: {total}')
    print(f'   - With Loan: {loans}')
    print(f'   - Percentage: {percentage:.2f}%\n')


� The main purpose of the code:

Analyze the relationship between family size and the probability of taking out a personal loan.
That is, you want to find out whether people with larger families take out more loans or not?

In [ ]:
cols1 = ['Family', 'Education', 'Personal Loan', 'Securities Account', 'CD Account', 'Online', 'CreditCard']
for col in cols1:
    print(f"\n{df[col].value_counts()}")
    print('_'*25)

In [ ]:
# Below code shows swarm plot for customers by Income and Education, seggregated by Personal Loan opted or not

sns.set(palette='icefire')
plt.figure(figsize=(15,5))
sns.swarmplot(data=df, x='Education', y='Income', hue='Personal Loan').set(title='Swarmplot: Education vs Income by Personal Loan\n0: Not Borrowed, 1: Borrowed');
plt.legend(loc="upper left" ,title="Opted Personal Loan", bbox_to_anchor=(1,1))

This plot clearly shows a positive relationship between:
 • Income
 • Education
 • and Likelihood of having a personal loan

In simple terms:

“Higher-educated, higher-income people are more likely to be approved for or opt into a personal loan.

In [ ]:
# Below code shows swarm plot for customers by Income and Family size, seggregated by Personal Loan opted or not

sns.set(palette='icefire')
plt.figure(figsize=(15,5))
sns.swarmplot(data=df, x='Family', y='Income', hue='Personal Loan').set(title='Swarmplot: Family size vs Income by Personal Loan\n0: Not Borrowed, 1: Borrowed');
plt.legend(loc="upper left" ,title="Opted Personal Loan", bbox_to_anchor=(1,1))

Income level still seems to be the most impactful factor in loan prediction
 • Family size shows only a mild pattern — perhaps slightly more loan-taking in families of size 3 or 4 with high income

In [ ]:
# Define columns that are numeric but actually categorical (binary or ordinal features)
categorical_like = ['CD Account',"CreditCard", "CreditCard","Education","Family", "Online" ,"Personal Loan"]


# Select only true continuous numerical features (exclude categorical-like ones)
true_numerical_cols = [
    col for col in df.select_dtypes(include=['int64', 'float64']).columns
    if col not in categorical_like
]

# Compute correlation matrix of only continuous features
corr_matrix = df[true_numerical_cols].corr()

# Convert to DataFrame for nicer display (optional)
corr_df = corr_matrix.reset_index().rename(columns={'index': 'Feature'})

# Display
print("Correlation matrix of true numerical features:")
print(corr_matrix)

In [ ]:
import pandas as pd

# numeric columns
numeric_df = df.select_dtypes(include='number')

#corrolation with target
correlations = numeric_df.corr()['Personal Loan'].sort_values(ascending=False)

# display
print("Correlation of features with target variable (Personal Loan):\n")
print(correlations)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# numerical columns
numeric_df = df.select_dtypes(include='number')

# calculate corrolation
corr_matrix = numeric_df.corr()


plt.figure(figsize=(12, 10))
sns.set(style='white')

# draw Heatmap
sns.heatmap(
    corr_matrix,
    annot=True,
    fmt=".2f",
    cmap='coolwarm',
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.8}
)

plt.title("Correlation Heatmap of Numerical Features", fontsize=16)
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
df

In [ ]:
num_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

In [ ]:
import pandas as pd
from scipy.stats import ttest_ind
import statsmodels.api as sm
import statsmodels.formula.api as smf

num_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
if 'Personal Loan' in num_cols:
    num_cols.remove('Personal Loan')


results = []


for col in num_cols:
    group0 = df[df['Personal Loan'] == 0][col]
    group1 = df[df['Personal Loan'] == 1][col]

    t_stat, p_val = ttest_ind(group0, group1, equal_var=False)  # Welch's t-test

    result = {
        'Feature': col,
        'T-Statistic': round(t_stat, 3),
        'P-Value': round(p_val, 5),
        'Significant (p < 0.05)': '✅ Yes' if p_val < 0.05 else '❌ No'
    }

    results.append(result)


results_df = pd.DataFrame(results)
print(results_df.sort_values('P-Value'))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind

num_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()

if "Personal Loan" in num_cols:
    num_cols.remove("Personal Loan")

results = []

for col in num_cols:
    group0 = df[df["Personal Loan"] == 0][col].dropna()
    group1 = df[df["Personal Loan"] == 1][col].dropna()

    if len(group0) > 0 and len(group1) > 0:
        t_stat, p_val = ttest_ind(group0, group1, equal_var=False)
        results.append({
            "Feature": col,
            "T_Statistic": round(t_stat, 4),
            "P_Value": round(p_val, 4)
        })

results_df = pd.DataFrame(results)
results_df["Color"] = np.where(results_df["P_Value"] < 0.05, "green", "red")

# Print check
print(results_df)

plt.figure(figsize=(12, 6))  
plt.bar(results_df["Feature"], results_df["P_Value"], color=results_df["Color"])
plt.axhline(0.05, color='black', linestyle='--', label="Significance Threshold (0.05)")
plt.title("T-Test: Personal Loan vs Numerical Features")
plt.xlabel("Feature")
plt.ylabel("P-Value")
plt.xticks(rotation=45)

plt.subplots_adjust(bottom=0.25)

plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
from scipy.stats import f_oneway

numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
categorical_cols = df.select_dtypes(include='object').columns.tolist()


potential_cat = ['Education', 'Family', 'CD Account', 'Online', 'CreditCard', 'Securities Account']
for col in potential_cat:
    if col in df.columns:
        df[col] = df[col].astype('object')
        if col not in categorical_cols:
            categorical_cols.append(col)
        if col in numeric_cols:
            numeric_cols.remove(col)


drop_cols = ['ID', 'ZIP Code']
numeric_cols = [col for col in numeric_cols if col not in drop_cols]


results = []
for cat_col in categorical_cols:
    if df[cat_col].nunique() < 2:
        continue
    for num_col in numeric_cols:
        try:

            groups = [df[df[cat_col] == level][num_col] for level in df[cat_col].unique()]
            f_stat, p_value = f_oneway(*groups)
            results.append({
                'Categorical Variable': cat_col,
                'Numeric Variable': num_col,
                'F-Statistic': round(f_stat, 3),
                'P-Value': round(p_value, 5),
                'Significant (<0.05)?': '✅ Yes' if p_value < 0.05 else '❌ No'
            })
        except:
            continue


anova_df = pd.DataFrame(results)


anova_df = anova_df.sort_values(by='P-Value')


pd.set_option('display.max_rows', None)
print(anova_df)

In [ ]:
df = df.drop(['ID'],axis=1)


In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations


num_features = ['Age', 'Experience', 'Income', 'CCAvg', 'Mortgage']


feature_pairs = list(combinations(num_features, 2))


for x_feat, y_feat in feature_pairs:
    plt.figure(figsize=(6, 4))
    sns.scatterplot(data=df, x=x_feat, y=y_feat, alpha=0.6, s=30)
    plt.title(f'Scatter Plot: {x_feat} vs {y_feat}', fontsize=14,color="red")
    plt.xlabel(x_feat,color="purple")
    plt.ylabel(y_feat,color="blue")
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations


num_features = ['Age', 'Experience', 'Income', 'CCAvg', 'Mortgage']


feature_pairs = list(combinations(num_features, 2))

plot_colors = sns.color_palette("husl", len(feature_pairs))


for idx, (x_feat, y_feat) in enumerate(feature_pairs):
    plt.figure(figsize=(6, 4))


    sns.scatterplot(
        data=df,
        x=x_feat,
        y=y_feat,
        hue='Personal Loan',
        palette={0: 'blue', 1: 'red'},
        alpha=0.6,
        s=40
    )

    plt.title(f'Scatter Plot: {x_feat} vs {y_feat}', fontsize=14, color=plot_colors[idx])
    plt.xlabel(x_feat)
    plt.ylabel(y_feat)
    plt.grid(True, linestyle='--', alpha=0.5)

    legend = plt.legend(
        title='Personal Loan',
        loc='best',
        fontsize=9,
        title_fontsize=10,
        fancybox=True,
        frameon=True,
        edgecolor='black',
        framealpha=0.9
    )

    legend.get_frame().set_linewidth(1.5)
    legend.get_frame().set_facecolor('#f9f9f9')

    plt.tight_layout()
    plt.show()

In [ ]:
plt = px.scatter(df, x="Age", y="Income",size="CCAvg", color ="Personal Loan",size_max=20,color_continuous_scale="Plasma",hover_data=df[['Education']],template = 'plotly_white')
plt.show()

Customers with higher income, are more likely to receive a loan

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (12,3))
plt.scatter(x=df['ZIP Code'],y=df['Personal Loan'],c="firebrick")
plt.annotate('maybe noise',xy=(9307,0),xytext=(15000,.4),arrowprops=dict(facecolor="red",shrink=3))
plt.grid()
plt.xlabel('ZIP Code')
plt.ylabel('Personal Loan')


In [ ]:
noise=df[(df['ZIP Code']<20000) & (df['Personal Loan']<.2)]
noise


In [ ]:
df.drop(index=[384],inplace=True)

In [ ]:
# List of numerical features
numerical_cols = ['Age', 'Experience', 'Income', 'CCAvg', 'Mortgage']

# Loop through each numerical feature and plot KDE (Density Plot)
for col in numerical_cols:
    plt.figure(figsize=(8, 4))
    sns.kdeplot(
        data=df,
        x=col,
        hue='Personal Loan',
        fill=True,
        alpha=0.7,          # Less transparency = more bold color
        linewidth=2,
        clip=(0, df[col].max()) # Thicker line
    )
    plt.title(f'Density Plot of {col} by Personal Loan Status')
    plt.xlabel(col)
    plt.ylabel('Density')
    plt.tight_layout()
    plt.show()

In [ ]:
palette_magma = sns.color_palette("magma", as_cmap=False, n_colors=2)
custom_palette = {0: palette_magma[0], 1: palette_magma[1]}
sns.pairplot(data=df, diag_kind='kde', hue='Personal Loan',palette=custom_palette)
plt.show()

This chart helps us understand which numerical features are useful for distinguishing between customers who took a loan and those who didn’t. It reveals patterns, relationships, and outliers that are important for building an effective prediction model

In [ ]:
# Create 2x2 subplots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1
sns.kdeplot(
    data=df, x='Age', y='Income', hue='Personal Loan',fontsize=24,color="red",
    fill=True, multiple='fill', alpha=0.8, palette='coolwarm', ax=axes[0, 0]
)
axes[0, 0].set_title('Age vs Income', fontsize=18,color="blue")
axes[0, 0].set_xlabel('Age', fontsize=18,color="red")
axes[0, 0].set_ylabel('Income', fontsize=18,color="purple")
axes[0, 0].tick_params(axis='both', labelsize=20)

# Plot 2
sns.kdeplot(
    data=df, x='Income', y='CCAvg', hue='Personal Loan',
    fill=True, multiple='fill', alpha=0.6, palette='coolwarm', ax=axes[0, 1]
)
axes[0, 1].set_title('Income vs CCAvg', fontsize=18,color="blue")
axes[0, 1].set_xlabel('Income', fontsize=18,color="red")
axes[0, 1].set_ylabel('CCAvg', fontsize=18,color="purple")
axes[0, 1].tick_params(axis='both', labelsize=20)

# Plot 3
sns.kdeplot(
    data=df, x='Age', y='Experience', hue='Personal Loan',
    fill=True, multiple='fill', alpha=0.6, palette='coolwarm', ax=axes[1, 0]
)
axes[1, 0].set_title('Age vs Experience', fontsize=18,color="blue")
axes[1, 0].set_xlabel('Age', fontsize=18,color="red")
axes[1, 0].set_ylabel('Experience', fontsize=18,color="purple")
axes[1, 0].tick_params(axis='both', labelsize=20)

# Plot 4
sns.kdeplot(
    data=df, x='Mortgage', y='Income', hue='Personal Loan',
    fill=True, multiple='fill', alpha=0.6, palette='coolwarm', ax=axes[1, 1]
)
axes[1, 1].set_title('Mortgage vs Income', fontsize=18,color="blue")
axes[1, 1].set_xlabel('Mortgage', fontsize=18,color="red")
axes[1, 1].set_ylabel('Income', fontsize=18,color="purple")
axes[1, 1].tick_params(axis='both', labelsize=20)

# Adjust legend font size
for ax in axes.flat:
    leg = ax.get_legend()
    if leg:

        for text in leg.get_texts():
            text.set_fontsize(20)


plt.tight_layout()
plt.show()

In [ ]:
nums=["Age","Experience","Income","CCAvg","Mortgage","ZIP Code"]
fig=plt.figure(figsize=(20,10))
for i,col in enumerate(nums):
    ax=fig.add_subplot(2,3,i+1)
    ax1=sns.distplot(df[col][df['Personal Loan']==0],hist=False, kde=True, label='No Personal Lone',color='purple')
    sns.distplot(df[col][df['Personal Loan']==1],hist=False, kde=True , ax=ax1,label='Personal Lone',color='red')
    ax.set_xlabel(col, fontsize=18,color="red")
    ax.set_ylabel("Density", fontsize=18,color="purple")
    ax.legend(fontsize=16)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, skewnorm

#
plt.figure(figsize=(15, 5))

# 1.negative skew
plt.subplot(1, 3, 1)
data_neg = skewnorm.rvs(a=-8, loc=0, scale=1, size=1000)
plt.hist(data_neg, bins=30, density=True, alpha=0.6, color='skyblue')
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, np.mean(data_neg), np.std(data_neg))
plt.plot(x, p, 'k', linewidth=2)
plt.title('Negatively Skewed',fontsize=18,color="blue")
plt.xlabel('X',fontsize=14,color="red")
plt.ylabel('Frequency',fontsize=14,color="purple")

# 2. normal distribution
plt.subplot(1, 3, 2)
data_norm = np.random.normal(loc=0, scale=1, size=1000)
plt.hist(data_norm, bins=30, density=True, alpha=0.6, color='lightgreen')
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, np.mean(data_norm), np.std(data_norm))
plt.plot(x, p, 'k', linewidth=2)
plt.title('Normal Distribution',fontsize=18,color="blue")
plt.xlabel('X',fontsize=14,color="red")
plt.ylabel('Frequency',fontsize=14,color="purple")

# 3. positive skew
plt.subplot(1, 3, 3)
data_pos = skewnorm.rvs(a=8, loc=0, scale=1, size=1000)
plt.hist(data_pos, bins=30, density=True, alpha=0.6, color='salmon')
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, np.mean(data_pos), np.std(data_pos))
plt.plot(x, p, 'k', linewidth=2)
plt.title('Positively Skewed',fontsize=18,color="blue")
plt.xlabel('X',fontsize=14,color="red")
plt.ylabel('Frequency',fontsize=14,color="purple")

plt.tight_layout()
plt.show()

In [ ]:
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

    

In [ ]:
# measure of skewness
df.skew(axis = 0, skipna = True)

In [ ]:
# Features to plot
features = ['Age', 'Experience']
custom_palette={0:"royalblue",1:"orange"}

for col in features:
    plt.figure(figsize=(30, 10), dpi=100)

    # Countplot with hue for Personal Loan
    sns.countplot(x=col, data=df, hue='Personal Loan', palette=custom_palette)

    # Axis settings
    plt.xlabel(col, fontsize=34)
    plt.ylabel(f'Count of {col}', fontsize=34)
    plt.xticks(rotation=90, fontsize=34)
    plt.yticks(fontsize=24)

    plt.grid(True)
    plt.title(f'Distribution of {col} by Personal Loan', fontsize=34,color="red")
    plt.legend(title='Personal Loan', fontsize=22, title_fontsize=22)
    plt.tight_layout()
    plt.show()

In [ ]:

df['Income_bin'] = pd.cut(df['Income'], bins=20)

plt.figure(figsize=(30, 10), dpi=100)
sns.countplot(x='Income_bin', data=df, hue='Personal Loan', palette={0: 'royalblue', 1: 'orange'})

plt.xlabel('Income Range', fontsize=34)
plt.ylabel('Count', fontsize=34)
plt.xticks(rotation=45, fontsize=34)
plt.yticks(fontsize=34)
plt.title('Distribution of Income by Personal Loan (Binned)', fontsize=38,color="orange")
plt.legend(title='Personal Loan', fontsize=20, title_fontsize=20)
plt.grid(True)
plt.tight_layout()
plt.show()

This chart helps banks or loan marketers target their offers better:
 • They might focus on customers earning $90K–$150K since those people are more likely to accept a personal loan

In [ ]:
columns1 = ['Age', 'Experience', 'Income','Family', 'CCAvg',
           'Education', 'Mortgage', 'Securities Account',
           'CD Account', 'Online', 'CreditCard']
df_mean = df.groupby('Personal Loan')[columns1].mean()
df_mean.plot(kind='barh', stacked=True, figsize=(12, 10), cmap="gist_heat",fontsize=24)

plt.xlabel('Average',fontsize=24,color="orange")
plt.ylabel("personal loan",color="red",fontsize=24)
plt.title('Average of each Feature VS Personal Loan',fontsize=34,color="red")
plt.legend(loc='lower right',fontsize=14)
plt.show()


1. Income:
The income of those who borrowed is significantly higher. This means that the bank usually lends to those who have more.
2. Credit Card Average Spending (CCAvg):
This number was also for those who borrowed. So, spending more on credit cards probably indicates active financial behavior that the bank likes.
3. CD Account:
Those who had this account borrowed much more. This means that the bank probably considered this account as a guarantee or an important privilege.
4. Mortgage:
Those who had a mortgage also had the highest level of mortgages in the group of borrowers. This means that having other loans (such as a mortgage) did not prevent them from getting a personal loan. It may even have helped!
5. Education:
Higher education is also more common in the group of borrowers. This means that more educated people have a better chance of getting a loan

In [ ]:
columns = ['Age', 'Experience', 'Income','Family', 'CCAvg',
           'Education', 'Mortgage', 'Securities Account',
           'CD Account', 'Online', 'CreditCard']
palette ="gist_heat"
for column in columns:
    plt.figure(figsize=(15,2))

    sns.violinplot(x=df[column], color="skyblue")
    plt.title(column,color="blue",fontsize=24)
    plt.xticks(fontsize=14,color="purple")
    plt.xlabel(column,fontsize=16,color="purple")
    plt.show()

Overall, customers with higher income, more education, and more interaction with various banking services had a higher chance of getting a loan.

In [ ]:
n = df.groupby('Personal Loan')[['Age', 'Experience', 'Income','Family', 'CCAvg',
           'Education', 'Mortgage', 'Securities Account',
           'CD Account', 'Online', 'CreditCard']].mean()


colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
          '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf', '#fcbf49']


ax = n.plot(kind='bar', figsize=(15, 10), color=colors, width=1.0)

plt.title("Average of Features for Personal Loan Groups", fontsize=24,color="orange")
plt.xlabel("Personal Loan (0 = No, 1 = Yes)", fontsize=24,color="purple")
plt.ylabel("Feature Averages", fontsize=18,color="red")
plt.xticks(fontsize=18, rotation=0, color='black')
plt.yticks(fontsize=18, rotation=0, color='black')


for i, container in enumerate(ax.containers):
    for bar in container:
        height = bar.get_height()
        if height > 0:
            ax.annotate(
                n.columns[i],
                xy=(bar.get_x() + bar.get_width()/2, height),
                xytext=(0,5),
                textcoords="offset points",
                ha='center', va='bottom',
                fontsize=20,
                color=colors[i],
                rotation=90
            )

plt.legend(loc='upper right', fontsize=14)
max_height = max([bar.get_height() for container in ax.containers for bar in container])

ax.set_ylim(0, max_height * 1.2)

plt.tight_layout()
plt.show()

In [ ]:
import plotly.express as px


categorical_cols = ['Education', 'Family', 'Online', 'CreditCard', 'CD Account', 'Securities Account']

for col in categorical_cols:
    fig = px.pie(df,
                 names=col,
                 title=f"<b style='color:crimson; font-size:20px;'>Distribution of {col}</b>",
                 hole=0.2,
                 width=600, height=500)

    fig.update_traces(textinfo='percent+label', textfont_size=16)
    fig.update_layout(title_x=0.5)
    fig.show()

In [ ]:
import plotly.express as px

# better label
df["Income_Label"] = df["Income"].apply(lambda x: f"Income: {x}")
df["Family_Label"] = df["Family"].apply(lambda x: f"Family: {x}")

fig = px.sunburst(
    df[df["Personal Loan"] == 1],  # just have personal loan
    path=["Income_Label", "Family_Label"],   
    values=[1]*len(df[df["Personal Loan"] == 1]),  
    title="Subburst of Approved Personal Loans by Income and Family Size",
    color="Income",  
    hover_data={"Income": True, "Family": True},
    width=1000,
    height=600
)

fig.update_layout(
    title_font=dict(size=22, color="darkblue"),
    margin=dict(t=50, l=25, r=25, b=25)
)

fig.show()

This pie chart shows the income groups in which borrowers are located and how many people in each income groups
and what size families they  are from.it gives you a very clear understanding of your customers loan behavior patterns.

In [ ]:
print(type(df))

In [ ]:
print(df.head())
print(df.columns)

In [ ]:
print ("credit card spending of customers who haven't received a loan:")
df[df['Personal Loan'] == 0]['CCAvg'].median()*100

In [ ]:
print ("credit card spending of customers who have received a loan:")
df[df['Personal Loan'] == 1]['CCAvg'].median()*100


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


df_c = ['Age', 'Experience', 'Income', 'ZIP Code', 'Mortgage']


sns.set_theme(style="white", palette="pastel")

for i in df_c:
    plt.figure(figsize=(8, 6))

    g = sns.JointGrid(data=df, x=i, y="Personal Loan", space=0)


    g.plot_joint(
        sns.kdeplot,
        fill=True,
        thresh=0.05,
        levels=30,
        cmap="coolwarm"
    )


    g.plot_marginals(
        sns.histplot,
        color="steelblue",
        alpha=0.8,
        bins=25
    )

    g.fig.suptitle(f"Relationship Between {i} and Personal Loan", fontsize=14)
    g.fig.tight_layout()
    g.fig.subplots_adjust(top=0.95)
    plt.show()

In [ ]:
df.dtypes

In [ ]:
#A MORE in-depth analysis(ZIP CODE)

In [ ]:
list_zipcode=list(df['ZIP Code'])



In [ ]:
df.drop(columns=["age_group","Income_bin","Income_Label","Family_Label"],inplace=True)

In [ ]:
df.head()

In [ ]:
import pgeocode

#American zipcode Database
nomi = pgeocode.Nominatim('us')


df['Latitude'] = df['ZIP Code'].apply(lambda x: nomi.query_postal_code(x).latitude)
df['Longitude'] = df['ZIP Code'].apply(lambda x: nomi.query_postal_code(x).longitude)
df['Place'] = df['ZIP Code'].apply(lambda x: nomi.query_postal_code(x).place_name)


In [ ]:
df.isna().sum()

In [ ]:
df[['Latitude', 'Longitude', 'Personal Loan']].corr()

Geographic location does not have a significant impact on loan acceptance in this data

In [ ]:
from scipy.stats import chi2_contingency
import pandas as pd

# (Contingency Table)
contingency = pd.crosstab(df['Place'], df['Personal Loan'])

 
chi2, p, dof, expected = chi2_contingency(contingency)


print(f"Chi2 = {chi2:.2f}, p-value = {p:.4f}")

In [ ]:
df.groupby('Place')['Personal Loan'].mean().sort_values(ascending=False)

The Bella Vista area has the highest loan acceptance rate

In [ ]:
df[df['Place']=='notfound']['ZIP Code'].value_counts()

In [ ]:
df['Place'].value_counts()

The largest data is from Los Angeles, which has 375 people

In [ ]:
df=df.astype({'Latitude':float,'Longitude':float})

In [ ]:
df[df['Latitude'].isna()]['ZIP Code'].unique()

This means we have 4 zip codes that have incomplete location information.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


top_places = df['Place'].value_counts().nlargest(10).index


filtered_df = df[df['Place'].isin(top_places)]


plt.figure(figsize=(14, 6))
sns.countplot(x='Place', hue='Personal Loan', data=filtered_df, palette='cool')


plt.title("Top 10 Places by Personal Loan Approval", fontsize=18,color="red")
plt.xlabel("Place", fontsize=16)
plt.ylabel("Count", fontsize=14)
plt.xticks(rotation=45, fontsize=14,color="purple")
plt.yticks(fontsize=12)
plt.legend(title="Personal Loan", loc='upper right')
plt.tight_layout()

plt.show()

In [ ]:
# Group by ZIP Code to calculate:
# 1. Average income of residents
# 2. Loan acceptance rate (percentage of people who got a loan)
zip_summary = df.groupby('ZIP Code').agg(
    Avg_Income=('Income', 'mean'),          # Average income
    Loan_Rate=('Personal Loan', 'mean')     # Loan rate (1=loan taken)
).reset_index()

# Convert loan rate to percentage
zip_summary['Loan_Rate'] = zip_summary['Loan_Rate'] * 100

# Sort by average income in descending order
zip_summary = zip_summary.sort_values(by='Avg_Income', ascending=False)

# Display the top ZIP Codes
import pandas as pd
pd.set_option('display.float_format', '{:.2f}'.format)
print(zip_summary.head(10))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


zip_summary = pd.DataFrame({
    'ZIP Code': ['91326', '91614', '91024', '96008', '91801'],
    'Avg_Income': [142.00, 140.25, 140.00, 139.00, 135.33],
    'Loan_Rate': [0.00, 25.00, 0.00, 66.67, 0.00]
})


plt.figure(figsize=(12, 6))
sns.scatterplot(
    data=zip_summary,
    x='Avg_Income',
    y='Loan_Rate',
    hue='ZIP Code',
    palette='Set2',
    s=200,
    edgecolor='black'
)


for i in range(zip_summary.shape[0]):
    plt.text(
        x=zip_summary['Avg_Income'][i] + 0.3,
        y=zip_summary['Loan_Rate'][i] + 0.5,
        s=zip_summary['ZIP Code'][i],
        fontsize=10
    )

plt.title('Loan Rate vs. Average Income by ZIP Code', fontsize=16, weight='bold')
plt.xlabel('Average Income (in $1000s)', fontsize=16,color="purple")
plt.ylabel('Loan Acceptance Rate (%)', fontsize=16,color="red")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import plotly.express as px


data = {
    'Prefix': list(range(10)),
    'States': [
        'MA, NY, NJ', 'PA, DE', 'VA, NC, SC', 'GA, AL, MS', 'MI, OH',
        'IA, MN', 'IL, IN', 'TX, OK', 'CO, UT', 'CA, WA'
    ],
    'Loan_Chance': ['Very High', 'Medium', 'Medium', 'Low', 'Medium',
                    'Medium', 'High', 'High', 'High', 'Very High'],
    'Reason': [
        'High income, strong credit', 
        'Mixed rural/urban', 
        'Mixed strength, variability',
        'High poverty, weaker credit',
        'Urban better, Detroit poor',
        'Lower income, fewer banks',
        'Chicago strong, rural weak',
        'Growing metros, good access',
        'Good income, active housing',
        'Wealthy, strong credit, competitive'
    ]
}

df_states = pd.DataFrame(data)


fig = px.bar(
    df_states,
    x='Prefix',
    y=df_states['Loan_Chance'].map({'Low': 0, 'Medium': 1, 'High': 2, 'Very High': 3}),
    color='Loan_Chance',
    text='States',
    hover_data=['Reason'],
    labels={'y': 'Loan Chance Score'},
    title='Estimated Loan Acceptance Potential by ZIP Prefix (US)'
)

fig.update_traces(textposition='outside')
fig.update_layout(
    yaxis=dict(
        tickmode='array',
        tickvals=[0, 1, 2, 3],
        ticktext=['Low', 'Medium', 'High', 'Very High']
    ),
    xaxis_title='ZIP Code Prefix (First Digit)',
    yaxis_title='Loan Chance',
    legend_title='Loan Level',
    height=600
)

fig.show()

In [ ]:
import pandas as pd
import plotly.express as px

data = {
    'Prefix': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    'States': [
        'MA, NY, NJ', 'PA, DE', 'VA, NC, SC', 'GA, AL, MS', 'MI, OH',
        'IA, MN', 'IL, IN', 'TX, OK', 'CO, UT', 'CA, WA'
    ],
    'Loan_Chance': [
        'Very High', 'Medium', 'Medium', 'Low', 'Medium',
        'Medium', 'High', 'High', 'High', 'Very High'
    ],
    'Reason': [
        'High income, strong credit',
        'Mixed income, some rural',
        'Diverse, some low-income',
        'High poverty, weak credit',
        'Urban/rural mix, Detroit weak',
        'Medium-low income, less banks',
        'Chicago strong, rural not much',
        'Big cities growing, good access',
        'Good income, active housing',
        'Wealthy, strong credit, active'
    ]
}

df_states = pd.DataFrame(data)


fig = px.bar(
    df_states,
    x='Prefix',
    y='Loan_Chance',
    color='Loan_Chance',
    text='States',
    hover_data=['States', 'Reason'],
    title='Loan Chance by ZIP Code Prefix and State',
    labels={'Loan_Chance': 'Loan Chance Level'},
    color_discrete_sequence=px.colors.qualitative.Prism
)

fig.update_traces(textposition='outside')
fig.update_layout(
    title_font=dict(size=24, color='darkblue'),
    xaxis_title='ZIP Code Prefix',
    yaxis_title='Loan Chance',
    margin=dict(t=80, l=40, r=40, b=80),
    height=600
)

fig.show()

In [ ]:
df.columns

In [ ]:
df = df.drop(['Latitude',"Place","Longitude"],axis=1)

In [ ]:
df.dtypes


In [ ]:
df[['ZIP Code', 'Personal Loan']].corr()

 <div style="background-color:rgba(0,217,255,0.1); padding:20px; border-radius:12px; margin: 20px 0;">
    <h3 style="color:#00d9ff;"> <code style=color:red>Before performing loan approval forecasting, we can consider several typical customer profiles that might qualify for a loan:</code></h3>
    <p>• Customers with moderate credit card spending and younger to mid-age, usually paired with limited work experience<br>
 • Applicants with a large mortgage and relatively young age, often having low experience<br>
 • Individuals with high income levels, but young and with short employment history.<br>
 • Some candidates may combine high income and low experience, regardless of age.<br>
 • Borrowers showing high credit card usage, low experience, and strong income levels.<br>
 • Profiles including high CCAvg, young age, and good income potential.<br>
 • Applicants with a significant mortgage, low age, and average income.<br>
 • Those with moderate mortgage and income, and mid-range credit card spending.<br>
 • Clients who have high mortgage, minimal experience, and moderate income.<br>
 • Lastly, customers with large mortgages, low experience, and low-to-moderate credit card spending.</p>
  </div>






<div style="
    background: linear-gradient(135deg,rgb(0, 0, 0),rgb(5, 19, 3),rgba(21, 105, 6, 0.83));
    width: 30%;
    margin: 40px auto;
    border-radius: 25px;
    text-align: center;
    box-shadow: 0px 0px 10px rgb(64, 255, 0);
    color: #e3f2fd;
    font-size: 24px;
    padding: 25px 35px;
    font-family: 'Poppins', 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    font-weight: 600;
    letter-spacing: 1px;
    transition: all 0.3s ease;
">
    Classification Models (Logistic, K-NN and Naïve Bayes)
</div>






In [ ]:
df.shape


In [ ]:
df.info()

Removing a column that has no effect on the final prediction

In [ ]:
df = df.drop(['ZIP Code'],axis=1)



<div style="
text-align:center;
width :fit-content;
  display: inline-block;
  padding: 10px 20px;
  background: linear-gradient(135deg, #667eea, #764ba2);
  color: white;
  font-family: 'Segoe UI', sans-serif;
  font-size: 25px;
  font-weight: bold;
  border-radius: 12px;
  box-shadow: 0 4px 12px rgba(0, 0, 0, 0.2);
  letter-spacing: 1px;
">
NORMALIZATION
</div>



In [ ]:
def Normalization(df,lst,rng) :
    scaler = preprocessing.MinMaxScaler(feature_range = rng)
    df[lst] = scaler.fit_transform(df[lst])
    return df

<div style="
    background: linear-gradient(135deg,rgb(0, 0, 0),rgb(5, 19, 3),rgba(21, 105, 6, 0.83));
    width: 50%;
    margin: 40px auto;
    border-radius: 25px;
    text-align: center;
    box-shadow: 0px 0px 10px rgb(64, 255, 0);
    color: #e3f2fd;
    font-size: 28px;
    padding: 25px 35px;
    font-family: 'Poppins', 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    font-weight: 600;
    letter-spacing: 1px;
    transition: all 0.3s ease;
">
    logistic Regresion Model
</div>


In [ ]:
X = pd.DataFrame (df , columns = ["Age" , "Experience" , "Income" ,"Family" , "CCAvg" , "Education" ,
                                    "Mortgage" , "Securities Account" , "CD Account" , "Online" , "CreditCard"])

y = df ["Personal Loan"].values.reshape (-1 , 1)

In [ ]:
df ["Personal Loan"].value_counts ()

In [ ]:
# training will be based on 70% of data and we split data to test_size=0.3 & train_size=0.7
X_train , X_test , y_train , y_test = train_test_split (X , y.ravel() , test_size = 0.3 , random_state = 0)


In [ ]:
print ('there are {} samples in train set and {} samples in test set.'.format (X_train.shape [0] , X_test.shape [0]))

In [ ]:
import numpy as np
import pandas as pd
from collections import Counter

class_counts = Counter(y_train)
print(class_counts)

counts = Counter(y_train)
total = len(y_train)  


weights = {cls: total / count for cls, count in counts.items()}

print(weights)


In [ ]:
from sklearn.model_selection import GridSearchCV
# defining the model & parameters
LR = LogisticRegression (max_iter = 5000 )
solvers = ['newton-cg' , 'lbfgs' , 'liblinear']
penalty = ["none" , "l1" , "l2" , "elasticnet"]
c_values = [100 , 10 , 1.0 , 0.1 , 0.01]
grid = dict (solver = solvers , penalty = penalty ,C = c_values) # defining grid search
grid_search = GridSearchCV (estimator = LR , param_grid= grid,n_jobs = -1, cv = 5, scoring = 'accuracy', error_score = 0)
grid_result = grid_search.fit (X_train , y_train)

# Summarizing results
print("The best score is: %f by using %s" % (grid_result.best_score_, grid_result.best_params_))

In [ ]:
# logistic regression
logreg = LogisticRegression (C = 100, penalty = 'l2' , solver = 'liblinear')
logreg.fit (X_train , y_train) # fiting the model
y_pred = logreg.predict (X_test) # predicting
y_predicted_proba = logreg.predict_proba (X_test)

In [ ]:
print("Accuracy", metrics.accuracy_score(y_test, y_pred))

In [ ]:
logreg.classes_

In [ ]:
logreg.intercept_

In [ ]:


logreg.coef_

In [ ]:
features = X.columns.tolist()
coefficients = logreg.coef_[0]

coef_df = pd.DataFrame({
    "Feature": features,
    "Coefficient": coefficients
})

coef_df.sort_values(by="Coefficient", ascending=False, inplace=True)
display(coef_df)

In [ ]:
print ("Accuracy: " , metrics.accuracy_score (y_test , y_pred))
print ("Score: " , logreg.score (X , y))
print ("F1 Score: ", f1_score (y_test , y_pred , average = "macro"))
print ("Precision Score: ", precision_score (y_test , y_pred , average = "macro"))
print ("Recall Score: ", recall_score (y_test , y_pred , average = "macro"))
print ("Mean Absolute Error: " , metrics.mean_absolute_error (y_test , y_pred))
print ("Mean Squared Error: " , metrics.mean_squared_error (y_test , y_pred))
print ("Root Mean Squared Error: " , np.sqrt (metrics.mean_squared_error (y_test , y_pred)))
print (classification_report (y_test , y_pred))

🎯 Final Conclusion

Class 0 (those who did not borrow) is predicted very well
• But class 1 (those who borrowed) has a lower accuracy and learning model (f1 = 0.68)
MY Logistic Regression model performs well and has high overall accuracy, but it performs poorly in identifying real borrowers (Class 1).

For projects like lending where Class 1 is more important (financial risk), you should increase the Recall and F1 Score of Class 1.

In [ ]:
confusion_matrix (y_test , y_pred)

🔍 Analysis:
 • True Negatives (1335): Excellent — model is great at identifying people who won’t take a loan.
 • True Positives (87): Decent — model correctly found 87 loan takers.
 • False Negatives (60): Bad — 60 people actually took loans, but the model failed to detect them.
⛔ Risky for banks, because missed real loan clients.
 • False Positives (18): Model incorrectly predicted loan for 18 people. Not great, but manageable

In [ ]:
# confusion matrix heatmap
plt.figure(figsize=(15, 8))
sns.set(font_scale=1.2)
sns.heatmap(confusion_matrix(y_test , y_pred) , annot=True, fmt="d", cmap="coolwarm", cbar=False,
            xticklabels=['Predict 0s', 'Predict 1s'], yticklabels=['Actual 0s', 'Actual 1s'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

TN = 1335: Model is very good at predicting non-loan customers.
 • TP = 87: Model successfully detected 87 real loan customers.
 • FP = 18: Only 18 false alarms (predicted loan, but they didn’t).
 • FN = 60: Missed 60 actual loan customers ❗ (this can be risky for a bank)

In [ ]:
!pip install scikit-plot

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt


y_scores = y_predicted_proba[:, 1]


fpr, tpr, thresholds = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)


plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC Curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')  # خط قرمز فرضی
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

In [ ]:

from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
y_pred_proba=logreg.predict_proba(X_test)

#classe 0
fpr0, tpr0, _ = roc_curve(1 - y_test, 1 - y_pred_proba[:, 1])
roc_auc0 = auc(fpr0, tpr0)

# class1
fpr1, tpr1, _ = roc_curve(y_test, y_pred_proba[:, 1])
roc_auc1 = auc(fpr1, tpr1)

plt.figure(figsize=(10, 6))

# classe1 line
plt.plot(fpr1, tpr1, label='Class 1 (AUC = %0.2f)' % roc_auc1, color='blue')

# class0 line
plt.plot(fpr0, tpr0, label='Class 0 (AUC = %0.2f)' % roc_auc0, color='red', linestyle='--')


plt.plot([0, 1], [0, 1], 'k--', label='Random Guess')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for Class 0 and Class 1')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

The graph has two ROC curves:
• Blue (Class 1): Indicates the model's ability to correctly identify Class 1 (those who have taken out a loan).
• Red dotted (Class 0): Indicates the model's ability to correctly identify Class 0 (those who have not taken out a loan).
• Gray diagonal line: Random Guess mode with AUC = 0.5.

⸻

🧠 Detailed analysis:

✅ AUC for Class 1 (blue): 0.96

This means that the model correctly identifies 96% of the time whether a person has actually taken out a loan or not.

✅ AUC for Class 0 (red): 0.96

This means that the model correctly identifies 96% of the time whether a person has not taken out a loan.

This is very good because the model has balanced performance — it performs well not only on the positive class, but also on the negative class

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

y_scores = y_predicted_proba[:, 1]


mask_pos = y_test == 1
mask_neg = y_test == 0


scores_pos = y_scores[mask_pos]
scores_neg = y_scores[mask_neg]


thresholds = np.linspace(0, 1, 500)


cum_pos = [np.mean(scores_pos <= t) for t in thresholds]
cum_neg = [np.mean(scores_neg <= t) for t in thresholds]


diffs = np.abs(np.array(cum_pos) - np.array(cum_neg))
ks_stat = np.max(diffs)
ks_idx = np.argmax(diffs)
ks_threshold = thresholds[ks_idx]


plt.figure(figsize=(12, 6))
plt.plot(thresholds, cum_neg, label='Class 0', color='navy')
plt.plot(thresholds, cum_pos, label='Class 1', color='brown')
plt.axvline(ks_threshold, color='black', linestyle='--',
            label=f'KS Statistic: {ks_stat:.3f} at {ks_threshold:.3f}')
plt.title('KS Statistic Plot')
plt.xlabel('Threshold')
plt.ylabel('Percentage below threshold')
plt.legend(loc='best')
plt.grid(True)
plt.tight_layout()
plt.show()

� Conclusion:

• This graph confirms that your model has a high ability to discriminate between classes 0 and 1.
• The value of KS = 0.802 shows that the model performance is excellent.
• You can use threshold = 0.09 as the optimal decision threshold

<div style="background-color:#1e1e2f; color:#f1f1f1; padding: 20px; border-radius: 16px; box-shadow: 0 0 20px rgba(0,0,0,0.6); font-family: 'Segoe UI', sans-serif; line-height: 1.7;">

  <h4 style="color:#00d9ff; text-align:center; border-bottom:2px solid #00d9ff; padding-bottom:10px;"> 
we must predict the target from following information:¶<br>ID: 5071<br>
Age: 42<br>
Experience: 16<br>
Income: 30<br>
ZIPCode: 92037 (REMOVED COLUMN!!!)<br>
Family: 3<br>
CCAvg: 1/2<br>
Education: Advanced or 3<br>
Mortgage: 0<br>
Personal Loan: ?<br>
Securities Account: 1<br>
CD Account: 0<br>
Online: 1<br>
CreditCard: 1<br>
Prediction Of Logistic Regression Model
</h4>


In [ ]:
df1 = df.copy ()
df2 = pd.DataFrame ({"Age" : [42] , "Experience" : [16] , "Income" : [30] ,"Family" : [3] ,
                    "CCAvg" : [1.2] , "Education" : [3] , "Mortgage" : [0] ,"Personal Loan" : [1] ,
                     "Securities Account" : [0] , "CD Account" : [0] , "Online" : [1] , "CreditCard" : [1]})
df2


In [ ]:
import pandas as pd
data2 = pd.concat([df1, df2])


In [ ]:
data2.shape

In [ ]:
X_train2 = data2 [["Age" , "Experience" , "Income" ,"Family" , "CCAvg" , "Education" ,
                                    "Mortgage" , "Securities Account" , "CD Account" , "Online" , "CreditCard"]][:4999]
y_train2= data2 ["Personal Loan"][:4999].values.reshape (-1 , 1)

In [ ]:
X_test2 = data2 [["Age" , "Experience" , "Income" ,"Family" , "CCAvg" , "Education" ,
                                    "Mortgage" , "Securities Account" , "CD Account" , "Online" , "CreditCard"]][4999:]

In [ ]:
X_test2

In [ ]:
logreg2 = LogisticRegression (C = 100 , penalty = 'l2' , solver = 'liblinear') # Defining model and parameters
logreg2.fit (X_train2 , y_train2)        # fit the model       
y_prediction = logreg2.predict(X_test2) 

if y_prediction == 0:
  print ("y_prediction is:" , y_prediction)
  print ("\33[41m Customer is not eligible to receive a bank loan.")
else:
  print ("y_prediction is:" , y_prediction)
  print ("\33[41m Customer is eligible to receive a bank loan.")


<div style="
    background: linear-gradient(135deg,rgb(0, 0, 0),rgb(5, 19, 3),rgba(21, 105, 6, 0.83));
    width: 50%;
    margin: 40px auto;
    border-radius: 25px;
    text-align: center;
    box-shadow: 0px 0px 10px rgb(64, 255, 0);
    color: #e3f2fd;
    font-size: 28px;
    padding: 25px 35px;
    font-family: 'Poppins', 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    font-weight: 600;
    letter-spacing: 1px;
    transition: all 0.3s ease;
">
    Naive Bayes Model
</div>


In [ ]:
# naive Bayes have 4 algorithms, we must examine all 4 of them
def NaiveBayes(X,y,test_size,model,stratify):
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0,stratify=stratify)
    Model = model()
    Model.fit(X_train,y_train.ravel())
    print('Accuracy of Model:' , np.round(metrics.accuracy_score(y_test,Model.predict(X_test)),3))
    Evaluation(Model,X,y,X_train,y_train,X_test,y_test)

In [ ]:
def Evaluation(Model,X,y,X_train,y_train,X_test,y_test) :
    cv_score = (cross_val_score(Model, X_train, y_train.ravel(), cv = StratifiedKFold(10), scoring='accuracy')).mean()
    print('Train Score : ',np.round(Model.score(X_train,y_train),2))
    print('Recall Score : ',np.round(recall_score(y_test,Model.predict(X_test)),2))
    print('F1 Score : ',np.round(f1_score(y_test,Model.predict(X_test)),2))
    print('Average Cross Validation Score : ',np.round(cv_score,2))
    print('jaccard Score : ',np.round(jaccard_score(y_test,Model.predict(X_test)),2))
    print('Precision Score : ',np.round(precision_score(y_test,Model.predict(X_test)),2))
    print('Confusion Matrix Results:')
    print(confusion_matrix(y_test,Model.predict(X_test)))
    plt.figure(figsize=(15, 8))
    sns.set(font_scale=1.2)
    sns.heatmap(confusion_matrix(y_test,Model.predict(X_test)) , annot=True, fmt="d", cmap="RdGy", cbar=False, xticklabels=['Predict 0s', 'Predict 1s'], yticklabels=['Actual 0s', 'Actual 1s'])
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

In [ ]:
df_Norm = df.copy()
features = ['Age','Experience','Income','Family',
           'CCAvg','Education','Mortgage','Securities Account','CD Account','Online'
           ,'CreditCard']
df_Norm = Normalization(df_Norm,features,(1,5))
x = df_Norm.drop('Personal Loan',axis=1)
y = df_Norm['Personal Loan'].values.reshape(-1,1)
print('Result of Gaussian Model : ')
NaiveBayes(x,y,0.1,GaussianNB,y)
print('Result of Bernoulli Model : ')
NaiveBayes(x,y,0.1,BernoulliNB,y)
print('Result of Complement Model : ')
NaiveBayes(x,y,0.1,ComplementNB,y)
print('Result of Multinomial Model : ')
NaiveBayes(x,y,0.1,MultinomialNB,y)

✅ Multinomial Naive Bayes performs best here because:
 • It made the fewest total errors.
 • It had zero false positives, which means no loans were mistakenly given to ineligible customers.
 • While recall (TP rate) is low, it prioritizes precision and risk minimization.

True Negatives (TN) = 1356
→ Correctly predicted people who didn’t get a loan.
 • False Positives (FP) = 0
→ Nobody was wrongly predicted to have received a loan when they didn’t.
 • False Negatives (FN) = 138
→ 138 people who actually got a loan were incorrectly predicted as not having received one.
 • True Positives (TP) = 6
→ Only 6 people who actually got loans were correctly predicted


<div style="
text-align:center;
width :fit-content;
  display: inline-block;
  padding: 10px 20px;
  background: linear-gradient(135deg, #667eea, #764ba2);
  color: white;
  font-family: 'Segoe UI', sans-serif;
  font-size: 25px;
  font-weight: bold;
  border-radius: 12px;
  box-shadow: 0 4px 12px rgba(0, 0, 0, 0.2);
  letter-spacing: 1px;
">
Prediction Of Naive Bayes Mode
</div>



In [ ]:
x_train2 = data2 [["Age" , "Experience" , "Income" ,"Family" , "CCAvg" , "Education" ,
                                    "Mortgage" , "Securities Account" , "CD Account" , "Online" , "CreditCard"]][:4999]
y_train2= data2 ["Personal Loan"][:4999].values.reshape (-1 , 1)

x_test2 = data2 [["Age" , "Experience" , "Income" ,"Family" , "CCAvg" , "Education" ,
                                    "Mortgage" , "Securities Account" , "CD Account" , "Online" , "CreditCard"]][4999:]
x_test2

model = MultinomialNB()
model.fit(x_train2, y_train2.ravel())
model.fit (x_train2 , y_train2) # fit the model       
y_prediction = model.predict(x_test2) 

if y_prediction == 0:
  print ("y_prediction is:" , y_prediction)
  print ("\33[41m Customer is not eligible to receive a bank loan.")
else:
  print ("y_prediction is:" , y_prediction)
  print ("\33[41m Customer is eligible to receive a bank loan.")
  

<div style="
    background: linear-gradient(135deg,rgb(0, 0, 0),rgb(5, 19, 3),rgba(21, 105, 6, 0.83));
    width: 30%;
    margin: 40px auto;
    border-radius: 25px;
    text-align: center;
    box-shadow: 0px 0px 10px rgb(64, 255, 0);
    color: #e3f2fd;
    font-size: 24px;
    padding: 25px 35px;
    font-family: 'Poppins', 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    font-weight: 600;
    letter-spacing: 1px;
    transition: all 0.3s ease;
">
    KNN Model
</div>


In [ ]:
def KNN(x,y,test_size,k,stratify) :
    TrainScore = []
    TestScore = []
    BestK = 0
    BestScore = 0
# test size=0.3
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=0,stratify=stratify)
    for i in range (1,k+1) :
        Model = KNeighborsClassifier(i)
        Model.fit(x_train,y_train.ravel())
        TrainScore.append(np.round(Model.score(x_train,y_train),3))
        TestScore.append(np.round(Model.score(x_test,y_test),3))
    BestK = TestScore.index(max(TestScore))+1
    BestScore = max(TestScore)
    Model = KNeighborsClassifier(BestK)
    Model.fit(x_train,y_train.ravel())
    print("Model Accuracy for K = %d is: "%k , np.round(TestScore,3))
    print('The best result is coming from k = %d with accuracy of = %s .'%(BestK,BestScore))
    Evaluation(Model,x,y,x_train,y_train,x_test,y_test)
    plt.plot(range(1,k+1),TrainScore,color='darkred',label='Train Accuracy')
    plt.plot(range(1,k+1),TestScore,color='red',label='Test Accuracy')
    plt.xlabel('k')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

<div style="
    background: linear-gradient(135deg,#fc00ff, #00dbde);
    width: 67%;
    margin: 40px auto;
    border-radius: 25px;
    text-align: center;
    box-shadow: 0 8px 20px rgba(0, 0, 0, 0.4);
    color: #e3f2fd;
    font-size: 28px;
    padding: 25px 35px;
    font-family: 'Poppins', 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    font-weight: 600;
    letter-spacing: 1px;
    border: 2px solidrgb(246, 100, 100);
    transition: all 0.3s ease;
">
  This function helps you:<br>
 • Try multiple values of k<br>
 • Choose the best one based on test performance<br>
 • Evaluate the model<br>
 • Visualize model accuracy vs. k<br>
</div>


In [ ]:
print('KNN Model Result:')
KNN(x,y,0.3,50,stratify=None)

The best performance of the KNN model was recorded for the value of k = 12

<span style="background-color: #1e1e2e; color: #ff0077ff; padding: 6px 14px; border-radius: 20px; font-weight: 600; font-size: 15px; font-family: 'Segoe UI', sans-serif; box-shadow: 0 0 8px rgba(0,255,213,0.4); display: inline-block;">
 The KNN model performs best with k = 1, achieving 97% accuracy
</span>



<span style="background-color: #1e1e2e; color: #ff0022ff; padding: 6px 14px; border-radius: 20px; font-weight: 600; font-size: 15px; font-family: 'Segoe UI', sans-serif; box-shadow: 0 0 8px rgba(0,255,213,0.4); display: inline-block;">
 The KNN model achieved the highest accuracy with a score of 0.97, making it the best-performing model<br>. Logistic Regression follows with an accuracy of 0.952, outperforming the Naive Bayes model. For KNN, the training accuracy is 1.0 when k = 1, but it decreases as the value of k increases. The test accuracy varies between 0.956 and 0.972, with the best score of 0.97 occurring at k = 12.
</span>



In [ ]:
df_norm = df.copy()
features2 = ['Age','Experience','Income','Family',
           'CCAvg','Education','Mortgage','Securities Account','CD Account','Online'
           ,'CreditCard']
df_norm = Normalization(df_norm,features2,(1,5))
x = df_norm.drop('Personal Loan',axis=1)
y = df_norm['Personal Loan'].values.reshape(-1,1)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=0,stratify=y)
print(classification_report(y_test , y_pred))
k = 1
Model = KNeighborsClassifier(k)
Model.fit(x_train,y_train.ravel())
y_pred = Model.predict(x_test)

plt.figure(figsize=(15, 8))
sns.set(font_scale=1.2)
sns.heatmap(confusion_matrix(y_test , y_pred) , annot=True, fmt="d", cmap="viridis", cbar=False,
            xticklabels=['Predict 0s', 'Predict 1s'], yticklabels=['Actual 0s', 'Actual 1s'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

<span style="background-color: #1e1e2e; color: #ff8800ff; padding: 6px 14px; border-radius: 20px; font-weight: 600; font-size: 15px; font-family: 'Segoe UI', sans-serif; box-shadow: 0 0 8px rgba(0,255,213,0.4); display: inline-block;">
35 customers did recevied a loan and it was predicted that they did not recevied a loan
</span>



<div style="background-color:#1e1e2f; color:#f1f1f1; padding: 30px; border-radius: 16px; box-shadow: 0 0 20px rgba(0,0,0,0.6);font-family: 'Segoe UI',sans-serif; line-height: 1.7;">

 
  <h4 style="color:#00d9ff; text-align:center; border-bottom:2px solid #00d9ff; padding-bottom:10px;">
  1346 customers did not recevied a loan and it was predicted that they did not received a loan<br>
  109 customers did recevied a loan and it was predicted that they recevied a loan<br>
  10 customers did not recevied a loan and it was predicted that they recevied a loan<br>
   </h4>

<div style="
    background: linear-gradient(135deg,rgb(0, 0, 0),rgb(5, 19, 3),rgba(21, 105, 6, 0.83));
    width: 30%;
    margin: 40px auto;
    border-radius: 25px;
    text-align: center;
    box-shadow: 0px 0px 10px rgb(64, 255, 0);
    color: #e3f2fd;
    font-size: 24px;
    padding: 25px 35px;
    font-family: 'Poppins', 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    font-weight: 600;
    letter-spacing: 1px;
    transition: all 0.3s ease;
">
    Prediction Of KNN Model
</div>


In [ ]:
X_train2 = data2 [["Age" , "Experience" , "Income" ,"Family" , "CCAvg" , "Education" ,
                                    "Mortgage" , "Securities Account" , "CD Account" , "Online" , "CreditCard"]][:4999].values
y_train2= data2 ["Personal Loan"][:4999].values.reshape (-1 , 1)
X_test2 = data2 [["Age" , "Experience" , "Income" ,"Family" , "CCAvg" , "Education" ,
                                    "Mortgage" , "Securities Account" , "CD Account" , "Online" , "CreditCard"]][4999:]
KNN2 = KNeighborsClassifier (n_neighbors = 7)
KNN2.fit (X_train2 , y_train2.ravel ())
y_prediction = KNN2.predict(X_test2) 
y_prediction
if y_prediction == 0:
  print ("y_prediction is:" , y_prediction)
  print ("\33[41m The customer is not eligible for the bank loan")
else:
  print ("y_prediction is:" , y_prediction)
  print ("\33[41m The customer is eligible for the bank loan")

In [ ]:
df_train = pd.DataFrame(x_train,columns=["Age" , "Experience" , "Income" ,"Family" , "CCAvg" , "Education" ,"Mortgage" , "Securities Account" , "CD Account" , "Online" , "CreditCard"])
df_train.insert(11,'Personal Loan',y_train)
df_test = pd.DataFrame(x_test,columns=['Age','Experience','Income','Family','CCAvg','Education','Mortgage',
                                         'Securities Account','CD Account','Online','CreditCard'])
df_test.insert(11,'Personal Loan',y_test)
df_pred = pd.DataFrame(x_test,columns=['Age','Experience','Income','Family','CCAvg','Education','Mortgage',
                                         'Securities Account','CD Account','Online','CreditCard'])
df_pred.insert(11,'Personal Loan',y_pred)

🎯 What is the overall purpose of these codes?
1. Display the output of the model for a specific data (whether the customer is eligible or not).
2. Prepare three DataFrames:
• df_train: data + training labels
• df_test: data + test labels
• df_pred: data + predicted labels
3. These DataFrames can be used for:
• Further analysis,
• Display in a table,
• Compare predictions with reality


<div style="
  text-align:center;
  width :fit-content;
  display: inline-block;
  padding: 10px 20px;
  background: linear-gradient(135deg, #667eea, #764ba2);
  color: white;
  font-family: 'Segoe UI', sans-serif;
  font-size: 25px;
  font-weight: bold;
  border-radius: 12px;
  box-shadow: 0 4px 12px rgba(0, 0, 0, 0.2);
  letter-spacing: 1px;
">
Improve Logistic Regression
</div>

In [ ]:
df["log_Income"]=np.log1p(df["Income"])
df["log_CCAvg"]=np.log1p(df["CCAvg"])
df["Income2"]=df["Income"]**2
df["age*in"]=df["Income"]*df["Age"]

In [ ]:
X = pd.DataFrame (df , columns = ["Age" , "Experience" , "Income" ,"Family" , "CCAvg" , "Education" ,
                                    "Mortgage" , "Securities Account" , "CD Account" , "Online" , "CreditCard","log_CCAvg",
"Income2",
"age*in",
"log_Income"])

y = df ["Personal Loan"].values.reshape (-1 , 1)
def Normalization(df,lst,rng) :
    scaler = preprocessing.MinMaxScaler(feature_range = rng)
    df[lst] = scaler.fit_transform(df[lst])
    return df
weights = {0: 3, 1: 7.0}

# training will be based on 70% of data and we split data to test_size=0.3 & train_size=0.7
X_train , X_test , y_train , y_test = train_test_split (X , y.ravel() , test_size = 0.3 , random_state = 0)

from sklearn.model_selection import GridSearchCV
# defining the model & parameters
LR = LogisticRegression (max_iter = 5000,class_weight=weights )
solvers = ['newton-cg' , 'lbfgs' , 'liblinear']
penalty = ["none" , "l1" , "l2" , "elasticnet"]
c_values = [100 , 10 , 1.0 , 0.1 , 0.01]
grid = dict (solver = solvers , penalty = penalty ,C = c_values) # defining grid search
grid_search = GridSearchCV (estimator = LR , param_grid= grid,n_jobs = -1, cv = 5, scoring = 'accuracy', error_score = 0)
grid_result = grid_search.fit (X_train , y_train)

# Summarizing results
print("The best score is: %f by using %s" % (grid_result.best_score_, grid_result.best_params_))



In [ ]:
# logistic regression
logreg = LogisticRegression (C = 0.1 , penalty = 'l2' , solver = 'newton-cg',class_weight=weights )
logreg.fit (X_train , y_train) # fiting the model
y_pred = logreg.predict (X_test) # predicting
y_predicted_proba = logreg.predict_proba (X_test)


print("Accuracy", metrics.accuracy_score(y_test, y_pred))



print ("Accuracy: " , metrics.accuracy_score (y_test , y_pred))
print ("Score: " , logreg.score (X , y))
print ("F1 Score: ", f1_score (y_test , y_pred , average = "macro"))
print ("Precision Score: ", precision_score (y_test , y_pred , average = "macro"))
print ("Recall Score: ", recall_score (y_test , y_pred , average = "macro"))
print ("Mean Absolute Error: " , metrics.mean_absolute_error (y_test , y_pred))
print ("Mean Squared Error: " , metrics.mean_squared_error (y_test , y_pred))
print ("Root Mean Squared Error: " , np.sqrt (metrics.mean_squared_error (y_test , y_pred)))
print (classification_report (y_test , y_pred))

In [ ]:
# confusion matrix heatmap
plt.figure(figsize=(15, 8))
sns.set(font_scale=1.2)
sns.heatmap(confusion_matrix(y_test , y_pred) , annot=True, fmt="d", cmap="coolwarm", cbar=False,
            xticklabels=['Predict 0s', 'Predict 1s'], yticklabels=['Actual 0s', 'Actual 1s'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

<div style="
text-align:center;
width :fit-content;
  display: inline-block;
  padding: 10px 20px;
  background: linear-gradient(135deg, #667eea, #764ba2);
  color: white;
  font-family: 'Segoe UI', sans-serif;
  font-size: 25px;
  font-weight: bold;
  border-radius: 12px;
  box-shadow: 0 4px 12px rgba(0, 0, 0, 0.2);
  letter-spacing: 1px;
">
Improve naive Bayes
</div>


In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    balanced_accuracy_score, confusion_matrix, classification_report,
    roc_auc_score, log_loss, jaccard_score
)

In [ ]:
from sklearn.utils.class_weight import compute_sample_weight
def NaiveBayes(X,y,test_size,model,stratify):
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0,stratify=stratify)
    Model = model()
    sample_weights = compute_sample_weight(class_weight={0: 0.3, 1: 0.7}, y=y_train)
    Model.fit(X_train,y_train.ravel(),sample_weight=sample_weights)
    print('Accuracy of Model:' , np.round(metrics.accuracy_score(y_test,Model.predict(X_test)),3))
    Evaluation(Model,X,y,X_train,y_train,X_test,y_test)


In [ ]:
def Evaluation(Model,X,y,X_train,y_train,X_test,y_test) :
    cv_score = (cross_val_score(Model, X_train, y_train.ravel(), cv = StratifiedKFold(10), scoring='accuracy')).mean()
    print('Train Score : ',np.round(Model.score(X_train,y_train),2))
    print('Recall Score : ',np.round(recall_score(y_test,Model.predict(X_test)),2))
    print('F1 Score : ',np.round(f1_score(y_test,Model.predict(X_test)),2))
    print('Average Cross Validation Score : ',np.round(cv_score,2))
    print('jaccard Score : ',np.round(jaccard_score(y_test,Model.predict(X_test)),2))
    print('Precision Score : ',np.round(precision_score(y_test,Model.predict(X_test)),2))
    print('Confusion Matrix Results:')
    print(confusion_matrix(y_test,Model.predict(X_test)))
    plt.figure(figsize=(15, 8))
    sns.set(font_scale=1.2)
    sns.heatmap(confusion_matrix(y_test,Model.predict(X_test)) , annot=True, fmt="d", cmap="RdGy", cbar=False, xticklabels=['Predict 0s', 'Predict 1s'], yticklabels=['Actual 0s', 'Actual 1s'])
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

In [ ]:
df_Norm = df.copy()


df_Norm = Normalization(df_Norm,features,(1,5))
X = df_Norm.drop('Personal Loan',axis=1)
y = df_Norm['Personal Loan'].values.reshape(-1,1)



print('Result of Gaussian Model : ')
NaiveBayes(X,y,0.1,GaussianNB,y)
print('Result of Bernoulli Model : ')
NaiveBayes(X,y,0.1,BernoulliNB,y)
print('Result of Complement Model : ')
NaiveBayes(X,y,0.1,ComplementNB,y)
print('Result of Multinomial Model : ')
NaiveBayes(X,y,0.1,MultinomialNB,y)

<div style="
text-align:center;
width :fit-content;
  display: inline-block;
  padding: 10px 20px;
  background: linear-gradient(135deg, #667eea, #764ba2);
  color: white;
  font-family: 'Segoe UI', sans-serif;
  font-size: 25px;
  font-weight: bold;
  border-radius: 12px;
  box-shadow: 0 4px 12px rgba(0, 0, 0, 0.2);
  letter-spacing: 1px;
">
Improve KNN
</div>


In [ ]:
X = pd.DataFrame (df , columns = ["Age" , "Experience" , "Income" ,"Family" , "CCAvg" , "Education" ,
                                    "Mortgage" , "Securities Account" , "CD Account" , "Online" , "CreditCard","log_CCAvg",
"Income2",
"age*in",
"log_Income"])

y = df ["Personal Loan"].values.reshape (-1 , 1)

In [ ]:
def KNN(X,y,test_size,k,stratify) :
    TrainScore = []
    TestScore = []
    BestK = 0
    BestScore = 0
# test size=0.3
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0,stratify=stratify)
    #Try diffrent values of K:
    for i in range (1,k+1) :
        Model = KNeighborsClassifier(i)
        Model.fit(X_train,y_train.ravel())
        TrainScore.append(np.round(Model.score(X_train,y_train),3))
        TestScore.append(np.round(Model.score(X_test,y_test),3))
        #Find the best K:
    BestK = TestScore.index(max(TestScore))+1
    BestScore = max(TestScore)
    #Train final model with best K:
    Model = KNeighborsClassifier(BestK)
    Model.fit(X_train,y_train.ravel())
    print("Model Accuracy for K = %d is: "%k , np.round(TestScore,3))
    print('The best result is coming from k = %d with accuracy of = %s .'%(BestK,BestScore))
    #Evaluate the model:
    Evaluation(Model,X,y,X_train,y_train,X_test,y_test)
    #plot accuracy for all K values:
    plt.plot(range(1,k+1),TrainScore,color='darkred',label='Train Accuracy')
    plt.plot(range(1,k+1),TestScore,color='red',label='Test Accuracy')
    plt.xlabel('k')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [ ]:
print('KNN Model Result:')
KNN(X,y,0.3,50,stratify=None)


In [ ]:
df_norm = df.copy()
features2= ["Age" , "Experience" , "Income" ,"Family" , "CCAvg" , "Education" ,
                                    "Mortgage" , "Securities Account" , "CD Account" , "Online" , "CreditCard","log_CCAvg",
"Income2",
"age*in",
"log_Income"]
df_norm = Normalization(df_norm,features2,(1,5))
X = df_norm.drop('Personal Loan',axis=1)
y = df_norm['Personal Loan'].values.reshape(-1,1)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0,stratify=y)
print(classification_report(y_test , y_pred))
k = 1
Model = KNeighborsClassifier(k)

Model.fit(X_train,y_train.ravel())
y_pred = Model.predict(X_test)

plt.figure(figsize=(15, 8))
sns.set(font_scale=1.2)
sns.heatmap(confusion_matrix(y_test , y_pred) , annot=True, fmt="d", cmap="viridis", cbar=False,
            xticklabels=['Predict 0s', 'Predict 1s'], yticklabels=['Actual 0s', 'Actual 1s'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
df.columns

<div style="
text-align:center;
width :fit-content;
  display: inline-block;
  padding: 10px 20px;
  background: linear-gradient(135deg, #667eea, #764ba2);
  color: white;
  font-family: 'Segoe UI', sans-serif;
  font-size: 25px;
  font-weight: bold;
  border-radius: 12px;
  box-shadow: 0 4px 12px rgba(0, 0, 0, 0.2);
  letter-spacing: 1px;
">
Ploting Actual Data VS Predicted Data
</div>


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


df_train = pd.DataFrame(X_train, columns=["Age", "Experience", "Income", "Family", "CCAvg", "Education", "Mortgage", "Securities Account", "CD Account", "Online", "CreditCard","log_CCAvg",
"Income2",
"age*in",
"log_Income"])
df_train['Personal Loan'] = y_train

df_test = pd.DataFrame(X_test, columns=["Age", "Experience", "Income", "Family", "CCAvg", "Education", "Mortgage", "Securities Account", "CD Account", "Online", "CreditCard","log_CCAvg",
"Income2",
"age*in",
"log_Income"])

df_test['Personal Loan (Actual)'] = y_test


df_pred = df_test.copy()
df_pred['Personal Loan (Predicted)'] = y_pred


print("🔹 Train Data:")
display(df_train.head())

print("🔹 Test Data with Actual Labels:")
display(df_test.head())

print("🔹 Test Data with Predicted Labels:")
display(df_pred.head())

plt.figure(figsize=(10, 4))
plt.plot(y_test[:50], label='Actual', marker='o')
plt.plot(y_pred[:50], label='Predicted', marker='x')
plt.title('Actual vs Predicted (First 50 Samples)')
plt.xlabel('Sample Index')
plt.ylabel('Loan Approval (0/1)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

df_test['Personal Loan'] = y_test
df_pred['Personal Loan'] = y_pred

rows = 5
cols = 2
colors = [['skyblue','navy'], ['lightgreen','green'], ['gold','darkorange'], ['orchid','purple'], ['tomato','firebrick']]

features = ['Income', 'Experience', 'Mortgage', 'Education', 'CCAvg'
]


for j, feature in enumerate(features):
    plt.figure(figsize=(14, 12))

    # Actual
    plt.subplot(rows, cols, j*2 + 1)
    sns.scatterplot(data=df_test, x=feature, y='Income', hue='Personal Loan', palette=colors[j])
    plt.title('Actual Data')

    if j == 2:
        plt.legend([],[], frameon=False)

    # Predicted
    plt.subplot(rows, cols, j*2 + 2)
    sns.scatterplot(data=df_pred, x=feature, y='Income', hue='Personal Loan', palette=colors[j])
    plt.title('Predicted Data')

    if j == 2:
        plt.legend([],[], frameon=False)

    plt.tight_layout()
    plt.show()

<div style="
    background: linear-gradient(135deg,rgb(0, 0, 0),rgb(5, 19, 3),rgba(105, 6, 92, 0.83));
    width: 30%;
    margin: 40px auto;
    border-radius: 25px;
    text-align: center;
    box-shadow: 0px 0px 10px rgba(255, 0, 212, 1);
    color: #e3f2fd;
    font-size: 24px;
    padding: 25px 75px;
    font-family: 'Poppins', 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    font-weight: 600;
    letter-spacing: 1px;
    transition: all 0.3s ease;
">
    This code is for comparing actual data and predicted by the machine learning model on a few important features like income, experience, etc
</div>

<div style="
  text-align:center;
  width :fit-content;
  display: inline-block;
  padding: 10px 20px;
  background: linear-gradient(135deg, #667eea, #764ba2);
  color: white;
  font-family: 'Segoe UI', sans-serif;
  font-size: 25px;
  font-weight: bold;
  border-radius: 12px;
  box-shadow: 0 4px 12px rgba(0, 0, 0, 0.2);
  letter-spacing: 1px;
">
Some data were predicted incorrectly
</div>

<div style="
    background: linear-gradient(135deg,rgb(39, 15, 15),rgba(219, 52, 105, 1),rgb(100, 44, 44));
    width: 67%;
    margin: 40px auto;
    border-radius: 25px;
    text-align: center;
    box-shadow: 0 8px 20px rgba(0, 0, 0, 0.4);
    color: #e3f2fd;
    font-size: 28px;
    padding: 25px 35px;
    font-family: 'Poppins', 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    font-weight: 600;
    letter-spacing: 1px;
    border: 2px solidrgb(246, 100, 100);
    transition: all 0.3s ease;
">
    calculate precision, recall, f1-score after improve models
</div>


<div style="
    background: linear-gradient(135deg,rgb(39, 15, 15),rgba(63, 52, 219, 1),rgb(100, 44, 44));
    width: 80%;
    margin: 40px auto;
    border-radius: 25px;
    text-align: center;
    box-shadow: 0 8px 20px rgba(0, 0, 0, 0.4);
    color: #e3f2fd;
    font-size: 28px;
    padding: 25px 35px;
    font-family: 'Poppins', 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    font-weight: 600;
    letter-spacing: 1px;
    border: 2px solidrgb(246, 100, 100);
    transition: all 0.3s ease;
">
    ✅ 1. Logistic Regression<br>

Class 0:<br>
 • Precision: 98%<br>
 • Recall: 98%<br>
 • F1-Score: 98%<br>

Class 1:<br>
 • Precision: 82%<br>
 • Recall: 81%<br>
 • F1-Score: 82%<br>

⸻

✅ 2. K-Nearest Neighbors (KNN)<br>

Class 0:<br>
 • Precision: 97%<br>
 • Recall: 1.00<br>
 • F1-Score: 98%<br>

Class 1:<br>
 • Precision: 98%<br>
 • Recall: 72%<br>
 • F1-Score: 83%<br>


</div>
